In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('./data/mturk.csv')
df.head(1)
# df.columns

In [ ]:
import itertools
from sklearn.metrics import cohen_kappa_score

def calc_cohens_kappa(df, subject, rater_amount=3):

    pivot_feature_df = df.pivot(index='HITId', columns='rater', values=subject)
    pivot_feature_df.columns = ['Worker_A', 'Worker_B', 'Worker_C']
    pivot_feature_df.reset_index(inplace=True)
    pivot_feature_df.drop(columns=["HITId"], inplace=True)
    pivot_feature_df.fillna(0, inplace=True)
    
    pivot_feature_df_transposed = pivot_feature_df.transpose()
    
    data = np.zeros((rater_amount, rater_amount))
    for j, k in list(itertools.combinations(range(rater_amount), r=2)):
        # table = to_table(pivot_feature_df_transposed.iloc[:, [j, k]], bins=3)
        # data[j, k] = cohens_kappa(table, return_results=False)
        data[j, k] = cohen_kappa_score(pivot_feature_df_transposed.iloc[j], pivot_feature_df_transposed.iloc[k])

        # rater_map = {"0-1": "C", "0-2": "B", "1-2": "A"}
        # df_filtered = df.loc[df['rater'] != rater_map[f"{j}-{k}"]]
        # data[j, k] = agreement_score(df_filtered, subject)
    return [data, pivot_feature_df]

import pingouin as pg

def calc_icc(df, subject):
    t_df = pd.melt(df, id_vars=['HITId', 'rater', "typicality", "plausibility", "complexity", "human_like_quality"])

    icc = pg.intraclass_corr(data=t_df, targets='HITId', raters='rater', ratings='complexity', nan_policy='omit')
    icc.set_index('Type')
    return icc


from agreement.utils.transform import pivot_table_frequency
from agreement.utils.kernels import (
    compute_weights, identity_kernel, linear_kernel, quadratic_kernel, ordinal_kernel,
    radical_kernel, radio_kernel, circular_kernel, bipolar_kernel
)
from agreement.metrics import (
    observed_agreement,
    s_score,
    cohens_kappa,
    gwets_gamma,
    krippendorffs_alpha,
    scotts_pi
)

def agreement_score(data, subject, method=cohens_kappa, kernel=identity_kernel):
    questions_answers_table = pivot_table_frequency(data["HITId"], data[subject])
    users_answers_table = pivot_table_frequency(data["rater"], data[subject])

    ob_ag = observed_agreement(questions_answers_table)
    score = method(questions_answers_table, kernel)

    if(method.__name__ == "cohens_kappa"):
        score=cohens_kappa(questions_answers_table, users_answers_table, kernel)

    return score, ob_ag

 # Data Analysis

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
# import plotly
import numpy as np

# changing plotting backend to either plotly
# pd.options.plotting.backend = "plotly"
# ..matplotlib
pd.options.plotting.backend = "matplotlib"
# or/and seaborn (requires matplotlib backend)

cmap=sns.color_palette("tab10",as_cmap=True)

sns.set()

#### Analyze Query Composition

In [ ]:
import re

def extract_query_composition(s):
    selected_column_amount = len([m.start() for m in re.finditer('(?=,)', s.split("\n")[0])]) + 1
    selected_aggregates = len([m.start() for m in re.finditer('(AVG|COUNT|MAX|MIN|SUM=,)', s.split("\n")[0])])
    column_amount = len(re.findall('^.*JOIN.*$', s, re.MULTILINE)) + 1
    group_by = len(re.findall('^.*GROUP.*$', s, re.MULTILINE))
    order_by = len(re.findall('^.*ORDER.*$', s, re.MULTILINE))
    where_line = re.findall('\n(WHERE.*)\n', s, re.MULTILINE)
    where_predicate_amount = len(re.findall('^.*OR|AND.*$', where_line[0], re.MULTILINE)) + 1 if where_line else 0
    having_line = re.findall('(HAVING.*)', s, re.MULTILINE)
    having_predicate_amount = len(re.findall('^.*OR|AND.*$', having_line[0], re.MULTILINE)) + 1 if having_line else 0

    return {
        "selected_column_amount": selected_column_amount,
        "selected_aggregates": selected_aggregates,
        "column_amount": column_amount,
        "where_predicate_amount": where_predicate_amount,
        "having_predicate_amount": having_predicate_amount,
        "group_by": group_by,
        "order_by": order_by
    }

df["Input.query_composition"] = df["Input.query"].apply(extract_query_composition)

#### Parallel Coordinates Graph

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

query_composition_df = pd.DataFrame.from_records(df.loc[::3,"Input.query_composition"])
query_composition_df["query"] = df.loc[::3,"Input.query"]
query_composition_df["average_complexity"]  = list(df.groupby("HITId")["complexity"].mean())
query_composition_df["rounded_average_complexity"] = query_composition_df["average_complexity"].apply(round)
query_composition_df["HITId"] = df.loc[::3,"HITId"]

fig = px.parallel_categories(query_composition_df, width= 1000, dimensions= [
    "selected_column_amount", 
                'selected_aggregates', 
                'column_amount',
                'where_predicate_amount',
                "having_predicate_amount",
                "group_by",
                "order_by"
                ],
    labels={
        "selected_column_amount": "Columns",
        "selected_aggregates": "Aggregates", 
        "column_amount": "Tables", 
        "where_predicate_amount": "Where", 
        "having_predicate_amount": "Having", 
        "group_by": "Group", 
        "order_by": "Order", 
        "rounded_average_complexity": ""
   },
   color="rounded_average_complexity", 
   color_continuous_scale=px.colors.sequential.Inferno,                  
)

fig.update_layout(
    font=dict(
        family="Courier New, monospace",
        size=17,  # Set the font size here
        color="Black"
        
    )
)

fig.update_yaxes(
    autorange=False,
    range = [-1,len(df)],
    tick0=0,
    dtick=1
)

fig.show()

#### Inter Rater Agreement (multi-rater) Analysis

In [ ]:
as_c = agreement_score(df, "complexity", krippendorffs_alpha, ordinal_kernel)
as_p = agreement_score(df, "plausibility", krippendorffs_alpha, ordinal_kernel)
as_t = agreement_score(df, "typicality", krippendorffs_alpha, ordinal_kernel)
as_h = agreement_score(df, "human_like_quality", krippendorffs_alpha, radio_kernel)

as_c, as_p, as_t, as_h

#### Similarity to SQL-Query
###### !! Requires GPU for fast computation - utilize cached computations

In [ ]:
# from sentence_transformers import SentenceTransformer

# model = SentenceTransformer("all-MiniLM-L6-v2")

# b_similarities = []
# n_similarities = []

# for i in df.index:
#   print(i)
#   sql_sentences = df.iloc[i]["Input.query"]
  
#   english_sentences = [df.iloc[i]["Input.baselineQuery"], df.iloc[i]["Input.nlQuery"]]

#   embeddings1 = model.encode(sql_sentences)
#   embeddings2 = model.encode(english_sentences)
#   similarities = model.similarity(embeddings1, embeddings2)
#   [b,n] = similarities[0]
#   b_similarities.append(b)
#   n_similarities.append(n)

# df["b_similarities"] = b_similarities
# df["n_similarities"] = n_similarities


# Load cached similarities
import re

df_with_similarities = pd.read_csv('./mturk_with_similarities.csv')
df["b_similarities"] = df_with_similarities["b_similarities"].apply(lambda x: float(re.match(r".*?\((\d*\.\d*)\)", x).groups()[0]))
df["n_similarities"] = df_with_similarities["n_similarities"].apply(lambda x: float(re.match(r".*?\((\d*\.\d*)\)", x).groups()[0]))

df["b_similarities"].mean(), df["n_similarities"].mean()

#### Readability

In [ ]:
import textstat

def calc_readability(df, method):
    return df["Input.baselineQuery"].apply(method), df["Input.nlQuery"].apply(method)
[baseline, nl] = calc_readability(df, textstat.flesch_reading_ease)

df["baselineQuery_Readability"] = baseline
df["nlQuery_Readability"] = nl
df["readability_dif"] = nl - baseline

#### Text Quality Metrics

In [ ]:
import spacy
import textdescriptives as td

nlp = spacy.load("en_core_web_lg")
nlp.add_pipe('textdescriptives/all')

def construct_spacy_docs(df, column):
    return df[column].apply(nlp)

df["query_doc"] = construct_spacy_docs(df, "Input.query")
df["baseline_doc"] = construct_spacy_docs(df, "Input.baselineQuery")
df["nl_doc"] = construct_spacy_docs(df, "Input.nlQuery")

In [ ]:
def quality_check(df, column):
    return df[column].apply(lambda x: x._.passed_quality_check)

df["baseline_text_quality"] = quality_check(df, "baseline_doc")
df["nl_text_quality"] = quality_check(df, "nl_doc")

df["baseline_text_quality"].value_counts(), df["nl_text_quality"].value_counts()

In [ ]:
def calc_dependency_distance(df, column):
    return df[column].apply(lambda x: x._.dependency_distance["dependency_distance_mean"])

df["baseline_dependency_distance"] = calc_dependency_distance(df, "baseline_doc")
df["nl_dependency_distance"] = calc_dependency_distance(df, "nl_doc")

df["baseline_dependency_distance"].mean(), df["nl_dependency_distance"].mean()

In [ ]:
def calc_coherence(df, column):
    return df[column].apply(lambda x: x._.coherence["first_order_coherence"])

df["baseline_coherence"] = calc_coherence(df, "baseline_doc")
df["nl_coherence"] = calc_coherence(df, "nl_doc")

df["baseline_coherence"].mean(), df["nl_coherence"].mean()

In [ ]:
from nltk.translate import chrf_score, bleu_score, gleu_score

def calc_translation_quality(df, method):
    func1 = lambda x: method(x["Input.query"], x["Input.baselineQuery"])
    func2 = lambda x: method(x["Input.query"], x["Input.nlQuery"])
    return df.apply(func1, axis=1), df.apply(func2, axis=1)
[baseline, nl] = calc_translation_quality(df, chrf_score.sentence_chrf)

df["baselineQuery_quality"] = baseline
df["nlQuery_quality"] = nl
df["quality_dif"] = baseline - nl

#### Typicality and Plausibility

In [ ]:
def filter_indecisive(x):
    return len(x.value_counts()) > 2

def indecisive_majority_vote(x):
    average_vote = x.mean()
    if average_vote >= 0:
        return average_vote.round()
    if average_vote < 0 and average_vote > -2:
        return -1
    else:
        return -2
    
def decisive_majority_vote(x): 
    votes = x.value_counts()
    return votes.idxmax()

indecisive_plausibility_HITs = df.groupby('HITId')['plausibility'].apply(filter_indecisive)
indecisive_plausibility_HIT_df = df[df["HITId"].isin(list(indecisive_plausibility_HITs[indecisive_plausibility_HITs == True].index))]
decisive_plausibility_HIT_df = df[df["HITId"].isin(list(indecisive_plausibility_HITs[indecisive_plausibility_HITs == False].index))]

indecisive_typicality_HITs = df.groupby('HITId')['typicality'].apply(filter_indecisive)
indecisive_typicality_HIT_df = df[df["HITId"].isin(list(indecisive_typicality_HITs[indecisive_typicality_HITs == True].index))]
decisive_typicality_HIT_df = df[df["HITId"].isin(list(indecisive_typicality_HITs[indecisive_typicality_HITs == False].index))]

# indecisive_plausibility_HIT_df.groupby('HITId')['plausibility'].apply(indecisive_majority_vote).value_counts(),indecisive_typicality_HIT_df.groupby('HITId')['plausibility'].apply(indecisive_majority_vote).value_counts()

indecisive_plausibility_HIT_df = indecisive_plausibility_HIT_df.groupby('HITId')['plausibility'].apply(indecisive_majority_vote).value_counts().to_frame().reset_index()
new_rows = pd.DataFrame({"plausibility": [2, -2], "count": [0,0]})
indecisive_plausibility_HIT_df = pd.concat([indecisive_plausibility_HIT_df, new_rows], ignore_index=True).sort_values("plausibility")

decisive_plausibility_HIT_df = decisive_plausibility_HIT_df.groupby('HITId')['plausibility'].apply(indecisive_majority_vote).value_counts().to_frame().reset_index()
new_rows = pd.DataFrame({"plausibility": [-2], "count": [0]})
decisive_plausibility_HIT_df = pd.concat([decisive_plausibility_HIT_df, new_rows], ignore_index=True).sort_values("plausibility")


indecisive_typicality_HIT_df = indecisive_typicality_HIT_df.groupby('HITId')['typicality'].apply(indecisive_majority_vote).value_counts().to_frame().reset_index()
new_rows = pd.DataFrame({"typicality": [2, -2], "count": [0,0]})
indecisive_typicality_HIT_df = pd.concat([indecisive_typicality_HIT_df, new_rows], ignore_index=True).sort_values("typicality")


decisive_typicality_HIT_df = decisive_typicality_HIT_df.groupby('HITId')['typicality'].apply(indecisive_majority_vote).value_counts().to_frame().reset_index()
new_rows = pd.DataFrame({"typicality": [-2], "count": [0]})
decisive_typicality_HIT_df = pd.concat([decisive_typicality_HIT_df, new_rows], ignore_index=True).sort_values("typicality")

In [ ]:
import matplotlib

indecisive_plausibility_HIT_df 
decisive_plausibility_HIT_df 
indecisive_typicality_HIT_df 
decisive_typicality_HIT_df

pos_mut_pcts = np.array([20, 10, 5, 7.5, 30, 50])
pos_cna_pcts = np.array([10, 0, 0, 7.5, 10, 0])
pos_both_pcts = np.array([10, 0, 0, 0, 0, 0])
neg_mut_pcts = np.array([10, 30, 5, 0, 10, 25])
neg_cna_pcts = np.array([5, 0, 7.5, 0, 0, 10])
neg_both_pcts = np.array([0, 0, 0, 0, 0, 10])
genes = ['PIK3CA', 'PTEN', 'CDKN2A', 'FBXW7', 'KRAS', 'TP53']
genes = decisive_typicality_HIT_df["typicality"]

# with sns.axes_style("white"):
#     sns.set_style("ticks")
#     sns.set_context("talk")
    
# plot details
bar_width = 0.35
line_width = 1
opacity = 0.7
pos_bar_positions = np.arange(len(indecisive_plausibility_HIT_df["count"]))
neg_bar_positions = pos_bar_positions + bar_width

# make bar plots
hpv_pos_mut_bar = plt.bar(pos_bar_positions, decisive_plausibility_HIT_df["count"], bar_width,
                            color='#ff7f0e',
                            edgecolor="grey",
                            linewidth=line_width,
                            label='Plausibility')
hpv_pos_cna_bar = plt.bar(pos_bar_positions, indecisive_plausibility_HIT_df["count"], bar_width,
                            bottom=decisive_plausibility_HIT_df["count"],
                            alpha=opacity,
                            color='#ff9f0e',
                            edgecolor="grey",
                            linewidth=line_width,
                            hatch='//',
                            label='Plausibility Indecisive')
hpv_neg_mut_bar = plt.bar(neg_bar_positions, decisive_typicality_HIT_df["count"], bar_width,
                            color='#1f77b4',
                            edgecolor="grey",
                            linewidth=line_width,
                            label='Typicality')
hpv_neg_cna_bar = plt.bar(neg_bar_positions, indecisive_typicality_HIT_df["count"], bar_width,
                            bottom=decisive_typicality_HIT_df["count"],
                            hatch='//',
                            color='#97cbee',
                            edgecolor="grey",
                            linewidth=line_width,
                            label='Typicality Indecisive')
plt.xticks(pos_bar_positions + bar_width/2, ["Very\nUnlikely", "Unlikely", "Neutral", "Likely", "Very\nLikely"])
plt.ylabel('Amount')
plt.legend(loc='best')

for i in range(cmap.N):
    rgba = cmap(i)
    # rgb2hex accepts rgb or rgba
    print(matplotlib.colors.rgb2hex(rgba))

#### Complexity

In [ ]:
data = df.groupby(["HITId"])["complexity"].mean().value_counts().sort_index()
ax = data.plot(kind="bar")

new_ticks = np.linspace(0, 9, 10)
ax.set_xticks(np.interp(new_ticks, data.index, np.arange(data.size)))
ax.set_xticklabels(new_ticks)

ax.figure.tight_layout()
ax.set_xlabel("Complexity")
ax.set_ylabel("Amount")

plt.xticks(rotation = 0)

#### Worker Distribution

In [ ]:
df["WorkerId"].value_counts().value_counts().sort_values(), len(df["WorkerId"].unique())